In [7]:
import numpy as np 
import matplotlib.pyplot as plt 
import os 
from tkinter import Tcl # for sorting the slices as in windows dir
from glob import glob 
from tqdm.notebook import tqdm 

from poreUtils import *


from PIL import Image

from scipy import ndimage as nd
from skimage.measure import label
#from skimage.feature import peak_local_max
from skimage.segmentation import watershed

import json 

In [2]:
# Using porespy 
import porespy as ps
import scipy.ndimage as spim
ps.visualization.set_mpl_style()
np.random.seed(1)

In [3]:
import pandas as pd

def extract_properties(list_of_r_instances):
    """
    Extract properties from a list of instances of 'r' and store them in a pandas dataframe.

    Parameters
    ----------
    list_of_r_instances : list
        List of instances of 'r'.
    Returns
    -------
    pandas.DataFrame
        Dataframe with columns for each property and rows for each instance of 'r'.
    """
    # Create a list of properties
    property_list = ['label', 'volume', 'bbox_volume', 'sphericity', 'surface_area', 'convex_volume',
                     'centroid', 'equivalent_diameter_area', 'euler_number', 'extent', 
                     'axis_major_length', 'axis_minor_length', 'solidity']

    # Create a dictionary to store the properties of the current instance of r
    properties = {}

    for prop in property_list:
        properties[prop] = []


    # Iterate over each instance of r
    for r_instance in list_of_r_instances:
        # Iterate over the property list
        for prop in property_list:
            try:
                # Extract the property from the current instance of r
                value = getattr(r_instance, prop)

                if prop == 'centroid':
                    # If the property is 'centroid', convert it to the nearest integer
                    value = np.round(value).astype(int).tolist()

                properties[prop].append(value)

            except AttributeError:
                # If the property does not exist, set it to None
                properties[prop] = None
            except NotImplementedError:
                # If there is a NotImplementedError, set it to None and continue to the next iteration
                properties[prop] = None
                continue

    # Create a pandas dataframe from the data list
    df = pd.DataFrame(properties)

    
    return df

In [4]:
# list all the folders where there rois 

# data dir 
data_dir = 'D:\\sagar\\Data'



In [5]:
# all the sample folder where there are rois 
sample_folders = os.listdir(data_dir)

all_rois = []
valid_folders = []
for d in sample_folders:
    roi_path = os.path.join(data_dir, d, 'roi')
    if (os.path.exists(roi_path)):
        valid_folders.append(os.path.join(data_dir, d,))
        all_rois += glob((roi_path +'\*'))
        


In [8]:
# Creating a separate folder to put porespy restult

for s in valid_folders:
    if not os.path.exists(os.path.join(s, 'porespy')):
        os.makedirs(os.path.join(s, 'porespy'))


bin_th = 55
dt_th = 12

processed_rois = []
faulty_rois = []

for roi_path in tqdm(all_rois):
    try:
        print('processing....', roi_path)
        
        tiffs = os.listdir(roi_path)
        slices = Tcl().call('lsort', '-dict', tiffs)
        vol = np.empty(shape=(300, 300, 300), dtype=np.uint8)
        for i, fname in enumerate(slices):
            im = Image.open(os.path.join(roi_path, fname))
            imarray = np.array(im)
            imarray = np.clip(imarray, 0.0005, 0.003)
            imarray = norm8bit(imarray)
            vol[i, :, :] = imarray

        th_vol = vol < bin_th
        th_vol = nd.binary_closing(th_vol, np.ones((3,3,3)))
        th_vol = th_vol.astype(np.uint8)
        th_vol = nd.binary_dilation(th_vol, np.ones((5,5,5)))
        th_vol = nd.binary_erosion(th_vol, np.ones((5,5,5)))

        dt3d = nd.distance_transform_edt(th_vol)
        
        mask = dt3d > dt_th
        markers = label(mask)
        labels = watershed(-dt3d, markers, mask=th_vol)

        props = ps.metrics.regionprops_3D(labels)

        df = extract_properties(props)
        features = df.to_dict(orient='list')
        features['bin_th'] = [bin_th]
        features['dt_th'] = [dt_th]
        features['dtmax'] = [dt3d.max()]
        

        jsonString = json.dumps(features)
        jsonFile = open(roi_path.replace('roi', 'porespy') + '_bth' + str(bin_th) + '_dth' + str(dt_th) + '.json', 'w')
        jsonFile.write(jsonString)
        jsonFile.close()
        processed_rois.append(roi_path)
        
    except:
        faulty_rois.append(roi_path)
        print('skipping....', roi_path)
        pass

print(faulty_rois)

  0%|          | 0/532 [00:00<?, ?it/s]

processing.... D:\sagar\Data\MD_1264_A10_Z6.6mm\roi\0-300x1050-1350x600-900
processing.... D:\sagar\Data\MD_1264_A10_Z6.6mm\roi\0-300x1450-1750x1200-1500
processing.... D:\sagar\Data\MD_1264_A10_Z6.6mm\roi\0-300x1450-1750x3200-3500
processing.... D:\sagar\Data\MD_1264_A10_Z6.6mm\roi\0-300x1450-1750x800-1100
processing.... D:\sagar\Data\MD_1264_A10_Z6.6mm\roi\1000-1300x1050-1350x1800-2100
processing.... D:\sagar\Data\MD_1264_A10_Z6.6mm\roi\1000-1300x1050-1350x2800-3100
processing.... D:\sagar\Data\MD_1264_A10_Z6.6mm\roi\1000-1300x1250-1550x1600-1900
processing.... D:\sagar\Data\MD_1264_A10_Z6.6mm\roi\1000-1300x1250-1550x3200-3500
processing.... D:\sagar\Data\MD_1264_A10_Z6.6mm\roi\1000-1300x1450-1750x1600-1900
processing.... D:\sagar\Data\MD_1264_A10_Z6.6mm\roi\1000-1300x1650-1950x1200-1500
processing.... D:\sagar\Data\MD_1264_A10_Z6.6mm\roi\1000-1300x1650-1950x600-900
processing.... D:\sagar\Data\MD_1264_A10_Z6.6mm\roi\1000-1300x1850-2150x1000-1300
processing.... D:\sagar\Data\MD_1264_

processing.... D:\sagar\Data\MD_1264_A1_1_Z3.3mm\roi\1000-1300x1400-1700x2000-2300
processing.... D:\sagar\Data\MD_1264_A1_1_Z3.3mm\roi\1000-1300x1600-1900x1600-1900
processing.... D:\sagar\Data\MD_1264_A1_1_Z3.3mm\roi\1000-1300x1800-2100x2400-2700
processing.... D:\sagar\Data\MD_1264_A1_1_Z3.3mm\roi\1000-1300x600-900x1200-1500
processing.... D:\sagar\Data\MD_1264_A1_1_Z3.3mm\roi\1000-1300x600-900x1400-1700
processing.... D:\sagar\Data\MD_1264_A1_1_Z3.3mm\roi\1000-1300x800-1100x1600-1900
processing.... D:\sagar\Data\MD_1264_A1_1_Z3.3mm\roi\1200-1500x1000-1300x1200-1500
processing.... D:\sagar\Data\MD_1264_A1_1_Z3.3mm\roi\1200-1500x1200-1500x1800-2100
processing.... D:\sagar\Data\MD_1264_A1_1_Z3.3mm\roi\1200-1500x1800-2100x2400-2700
processing.... D:\sagar\Data\MD_1264_A1_1_Z3.3mm\roi\1200-1500x2000-2300x2400-2700
processing.... D:\sagar\Data\MD_1264_A1_1_Z3.3mm\roi\1200-1500x600-900x1400-1700
processing.... D:\sagar\Data\MD_1264_A1_1_Z3.3mm\roi\1200-1500x800-1100x1200-1500
processing..

processing.... D:\sagar\Data\MD_1264_A3_1_Z3.3mm\roi\0-300x1200-1500x1200-1500
processing.... D:\sagar\Data\MD_1264_A3_1_Z3.3mm\roi\0-300x1200-1500x200-500
processing.... D:\sagar\Data\MD_1264_A3_1_Z3.3mm\roi\0-300x1200-1500x600-900
processing.... D:\sagar\Data\MD_1264_A3_1_Z3.3mm\roi\0-300x1400-1700x1200-1500
processing.... D:\sagar\Data\MD_1264_A3_1_Z3.3mm\roi\0-300x2000-2300x400-700
processing.... D:\sagar\Data\MD_1264_A3_1_Z3.3mm\roi\1000-1300x1000-1300x800-1100
processing.... D:\sagar\Data\MD_1264_A3_1_Z3.3mm\roi\1200-1500x1200-1500x600-900
processing.... D:\sagar\Data\MD_1264_A3_1_Z3.3mm\roi\1200-1500x1400-1700x600-900
processing.... D:\sagar\Data\MD_1264_A3_1_Z3.3mm\roi\1400-1700x1600-1900x600-900
processing.... D:\sagar\Data\MD_1264_A3_1_Z3.3mm\roi\200-500x1000-1300x1600-1900
processing.... D:\sagar\Data\MD_1264_A3_1_Z3.3mm\roi\200-500x1200-1500x1000-1300
processing.... D:\sagar\Data\MD_1264_A3_1_Z3.3mm\roi\200-500x1200-1500x200-500
processing.... D:\sagar\Data\MD_1264_A3_1_Z3.

processing.... D:\sagar\Data\MD_1264_A4_1_Z0.0mm_Z3.3mm_corr_phrt\roi\1200-1500x800-1100x2100-2400
processing.... D:\sagar\Data\MD_1264_A4_1_Z0.0mm_Z3.3mm_corr_phrt\roi\1400-1700x1000-1300x1700-2000
processing.... D:\sagar\Data\MD_1264_A4_1_Z0.0mm_Z3.3mm_corr_phrt\roi\1400-1700x1200-1500x1500-1800
processing.... D:\sagar\Data\MD_1264_A4_1_Z0.0mm_Z3.3mm_corr_phrt\roi\1400-1700x1200-1500x1700-2000
processing.... D:\sagar\Data\MD_1264_A4_1_Z0.0mm_Z3.3mm_corr_phrt\roi\1400-1700x1400-1700x1700-2000
processing.... D:\sagar\Data\MD_1264_A4_1_Z0.0mm_Z3.3mm_corr_phrt\roi\200-500x1000-1300x1300-1600
processing.... D:\sagar\Data\MD_1264_A4_1_Z0.0mm_Z3.3mm_corr_phrt\roi\200-500x1200-1500x1300-1600
processing.... D:\sagar\Data\MD_1264_A4_1_Z0.0mm_Z3.3mm_corr_phrt\roi\200-500x800-1100x1500-1800
processing.... D:\sagar\Data\MD_1264_A4_1_Z0.0mm_Z3.3mm_corr_phrt\roi\200-500x800-1100x2100-2400
processing.... D:\sagar\Data\MD_1264_A4_1_Z0.0mm_Z3.3mm_corr_phrt\roi\400-700x1000-1300x1300-1600
processing...

processing.... D:\sagar\Data\MD_1264_A6_1_Z3.3mm_corr_phrt\roi\200-500x1450-1750x1200-1500
processing.... D:\sagar\Data\MD_1264_A6_1_Z3.3mm_corr_phrt\roi\200-500x1450-1750x2400-2700
processing.... D:\sagar\Data\MD_1264_A6_1_Z3.3mm_corr_phrt\roi\200-500x1650-1950x1400-1700
processing.... D:\sagar\Data\MD_1264_A6_1_Z3.3mm_corr_phrt\roi\200-500x1650-1950x3000-3300
processing.... D:\sagar\Data\MD_1264_A6_1_Z3.3mm_corr_phrt\roi\200-500x1850-2150x3000-3300
processing.... D:\sagar\Data\MD_1264_A6_1_Z3.3mm_corr_phrt\roi\200-500x2250-2550x3200-3500
processing.... D:\sagar\Data\MD_1264_A6_1_Z3.3mm_corr_phrt\roi\400-700x1050-1350x1400-1700
processing.... D:\sagar\Data\MD_1264_A6_1_Z3.3mm_corr_phrt\roi\400-700x1050-1350x2600-2900
processing.... D:\sagar\Data\MD_1264_A6_1_Z3.3mm_corr_phrt\roi\400-700x1250-1550x1200-1500
processing.... D:\sagar\Data\MD_1264_A6_1_Z3.3mm_corr_phrt\roi\400-700x1250-1550x1600-1900
processing.... D:\sagar\Data\MD_1264_A6_1_Z3.3mm_corr_phrt\roi\400-700x1450-1750x1400-1700

processing.... D:\sagar\Data\MD_1264_B2_1\roi\0-300x2100-2400x1950-2250
processing.... D:\sagar\Data\MD_1264_B2_1\roi\1000-1300x1700-2000x1950-2250
processing.... D:\sagar\Data\MD_1264_B2_1\roi\1200-1500x900-1200x1950-2250
processing.... D:\sagar\Data\MD_1264_B2_1\roi\1400-1700x1500-1800x1350-1650
processing.... D:\sagar\Data\MD_1264_B2_1\roi\200-500x2100-2400x1750-2050
processing.... D:\sagar\Data\MD_1264_B2_1\roi\400-700x1900-2200x1750-2050
processing.... D:\sagar\Data\MD_1264_B2_1\roi\800-1100x1900-2200x1750-2050
processing.... D:\sagar\Data\MD_1264_B2_1\roi\800-1100x2100-2400x1750-2050
processing.... D:\sagar\Data\MD_1264_B3_1_Z3.3mm\roi\1000-1300x1000-1300x3100-3400
processing.... D:\sagar\Data\MD_1264_B3_1_Z3.3mm\roi\1400-1700x1000-1300x2900-3200
processing.... D:\sagar\Data\MD_1264_B3_1_Z3.3mm\roi\1400-1700x1600-1900x1100-1400
processing.... D:\sagar\Data\MD_1264_B3_1_Z3.3mm\roi\1400-1700x1600-1900x1900-2200
processing.... D:\sagar\Data\MD_1264_B4_1_Z3.3mm\roi\0-300x1300-1600x15